<a href="https://colab.research.google.com/github/sakigo9/MY_ML_CODE/blob/master/IMDB_DATASET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

import tensorflow as tf
print(tf.__version__)


#For tensorflow having version<2.x.
tf.enable_eager_execution()

# !pip install -q tensorflow-datasets

1.14.0


In [0]:
import tensorflow_datasets as tfds
#We took imbd dataset for evaluation.
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


In [0]:
import numpy as np
#We took datsets and divded them into training and testing cases.
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [0]:
#Using variables for storing parameters values.
#16 Dimentional
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)



In [10]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[20]))
print(training_sentences[20])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? b this movie is inspiring to anyone who is or has been in a tough jam whether <OOV> or emotionally you will definitely laugh which is the best medicine left in a bad financial situation when her husband dies grace has to find a new way to make some money and it's not exactly legal which adds to the humour even my boyfriend liked it so don't think that it's a chick <OOV>
b"This movie is inspiring to anyone who is or has been in a tough jam, whether financially or emotionally. You will definitely laugh, which is the best medicine! :) Left in a bad financial situation when her husband dies, Grace has to find a new way to make some money and it's not exactly legal which adds to the humour. Even my boyfriend liked it so don't think that it's a chick-flic."


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [0]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

W0703 10:53:43.260837 140029060700032 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 269us/sample - loss: 0.5012 - acc: 0.7328 - val_loss: 0.3822 - val_acc: 0.8265
Epoch 2/10
25000/25000 [==============================] - 5s 219us/sample - loss: 0.2342 - acc: 0.9102 - val_loss: 0.3798 - val_acc: 0.8343
Epoch 3/10
25000/25000 [==============================] - 5s 214us/sample - loss: 0.0778 - acc: 0.9818 - val_loss: 0.4606 - val_acc: 0.8234
Epoch 4/10
25000/25000 [==============================] - 5s 214us/sample - loss: 0.0165 - acc: 0.9986 - val_loss: 0.5539 - val_acc: 0.8204
Epoch 5/10
25000/25000 [==============================] - 5s 214us/sample - loss: 0.0038 - acc: 0.9999 - val_loss: 0.5907 - val_acc: 0.8273
Epoch 6/10
25000/25000 [==============================] - 5s 215us/sample - loss: 0.0015 - acc: 1.0000 - val_loss: 0.6366 - val_acc: 0.8267
Epoch 7/10
25000/25000 [==============================] - 5s 216us/sample - loss: 0.0011 - acc: 0.9999 - val_l

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [0]:


try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

In [0]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1430], [968], [4], [1537], [1537], [4744], [], [790], [2016], [11], [2923], [2191], [], [790], [2016], [11], [579], [], [11], [579], [], [4], [1782], [4], [4510], [11], [2923], [1276], [], [], [2016], [1005], [2923], [968], [579], [790], []]
